# Análisis de balance de clases: Etapa 1 y Etapa 2
# Este notebook evalúa:
# - Distribución de `existe_replica_fuerte` (Etapa 1)
# - Distribución de `ventana_temporal_replica` (Etapa 2)
# - Sub-distribuciones por magnitud y zona
# - Sugerencias de técnicas de balanceo


In [2]:
import pandas as pd
import numpy as np

# Cargar dataset de features de la segunda etapa
df = pd.read_csv('../2da_etapa_creacion_features/seismic_features.csv')

print(f"Shape total: {df.shape}")
print("Columnas:", list(df.columns))

# Verificar presencia de columnas clave
required_cols = ['existe_replica_fuerte','ventana_temporal_replica','es_mainshock','Magnitude']
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f"Faltan columnas requeridas: {missing}")

# Filtrar solo mainshocks para análisis principal
main_df = df[df['es_mainshock'] == 1].copy()
print(f"Mainshocks (es_mainshock==1): {len(main_df)}")
display(main_df.head(10))

Shape total: (4015, 29)
Columnas: ['id_sismo_principal', 'Date(UTC)', 'Latitude', 'Longitude', 'Depth', 'Magnitude', 'celda_geografica', 'zona_sismica', 'distancia_a_costa_km', 'estacion_año', 'es_sismo_somero', 'intensidad_categoria', 'energia_liberada_estimada', 'es_mainshock', 'actividad_M5_15d', 'actividad_M6_30d', 'actividad_M7_90d', 'actividad_reciente_completa', 'brecha_magnitud_zona', 'sismos_previos_celda', 'densidad_sismica_zona', 'ratio_replicas_24h', 'ratio_replicas_48h', 'ratio_replicas_72h', 'magnitud_umbral', 'existe_replica_fuerte', 'ventana_temporal_replica', 'similitud_promedio_vecinos', 'conflicto_modelos']
Mainshocks (es_mainshock==1): 236


,id_sismo_principal,Date(UTC),Latitude,Longitude,Depth,Magnitude,celda_geografica,zona_sismica,distancia_a_costa_km,estacion_año,...,sismos_previos_celda,densidad_sismica_zona,ratio_replicas_24h,ratio_replicas_48h,ratio_replicas_72h,magnitud_umbral,existe_replica_fuerte,ventana_temporal_replica,similitud_promedio_vecinos,conflicto_modelos
0,2012-03-03T11:01:47_-30.19_-71.45,2012-03-03 11:01:47,-30.19,-71.45,35,5.6,-30_-71,Centro,37.555688,Otoño,...,0,0.000000,0.0,0.0,0.0,4.6,0,4,NaN,NaN
5,2012-03-25T22:37:06_-35.2_-72.22,2012-03-25 22:37:06,-35.20,-72.22,41,6.8,-35_-72,Centro,194.990892,Otoño,...,0,0.000000,0.0,0.0,0.0,5.8,0,5,NaN,NaN
7,2012-04-30T07:39:44_-29.8_-71.64,2012-04-30 07:39:44,-29.80,-71.64,43,6.0,-30_-72,Norte,39.221898,Otoño,...,0,0.000000,0.0,0.0,0.0,5.0,0,4,NaN,NaN
8,2012-05-14T10:00:40_-18.11_-70.24,2012-05-14 10:00:40,-18.11,-70.24,120,6.4,-18_-70,Norte,44.893891,Otoño,...,0,0.000000,0.0,0.0,0.0,5.4,0,4,NaN,NaN
9,2012-05-19T08:35:09_-25.74_-70.86,2012-05-19 08:35:09,-25.74,-70.86,84,6.1,-26_-71,Norte,236.997427,Otoño,...,0,0.000000,0.0,0.0,0.0,5.1,0,4,NaN,NaN
10,2012-08-07T00:39:03_-27.88_-70.58,2012-08-07 00:39:03,-27.88,-70.58,74,5.5,-28_-71,Norte,233.891630,Invierno,...,0,0.000000,0.0,0.0,0.0,4.5,0,4,NaN,NaN
14,2012-10-08T01:50:25_-21.83_-68.54,2012-10-08 01:50:25,-21.83,-68.54,121,5.7,-22_-69,Norte,278.090641,Primavera,...,0,0.000000,0.0,0.0,0.0,4.7,0,4,NaN,NaN
15,2012-10-11T17:22:10_-32.88_-70.65,2012-10-11 17:22:10,-32.88,-70.65,95,5.7,-33_-71,Centro,93.360358,Primavera,...,1,0.000097,0.0,0.0,0.0,4.7,0,4,NaN,NaN
17,2012-11-14T19:02:03_-29.24_-71.23,2012-11-14 19:02:03,-29.24,-71.23,82,5.7,-29_-71,Norte,73.414136,Primavera,...,0,0.000000,0.0,0.0,0.0,4.7,0,4,NaN,NaN
20,2013-01-13T21:23:27_-20.12_-69.31,2013-01-13 21:23:27,-20.12,-69.31,90,5.5,-20_-69,Norte,210.614984,Verano,...,2,0.000173,0.0,0.0,0.0,4.5,0,4,NaN,NaN


In [3]:
# Etapa 1: Distribución de existe_replica_fuerte en mainshocks
stage1_counts = main_df['existe_replica_fuerte'].value_counts().rename('count')
stage1_percent = (stage1_counts / stage1_counts.sum() * 100).round(2).rename('percent')
stage1_tbl = pd.concat([stage1_counts, stage1_percent], axis=1)
print("Distribución Etapa 1 (mainshocks):")
print(stage1_tbl)

positive_rate = stage1_counts.get(1, 0) / stage1_counts.sum()
print(f"\nTasa positiva (réplica fuerte ≤72h): {positive_rate:.3f}")

# Ratio de positivos por bin de magnitud (para ver correlación con magnitud)
bins = [5.5,5.8,6.1,6.4,6.7,7.0,8.0,10.0]
main_df['mag_bin'] = pd.cut(main_df['Magnitude'], bins=bins, include_lowest=True)
mag_grp = main_df.groupby('mag_bin')['existe_replica_fuerte'].agg(['count','mean']).rename(columns={'count':'eventos','mean':'tasa_positiva'})
print("\nTasa positiva por bin de magnitud:")
print(mag_grp)
mag_grp.head()

Distribución Etapa 1 (mainshocks):
                       count  percent
existe_replica_fuerte                
0                        211    89.41
1                         25    10.59

Tasa positiva (réplica fuerte ≤72h): 0.106

Tasa positiva por bin de magnitud:
              eventos  tasa_positiva
mag_bin                             
(5.499, 5.8]      132       0.075758
(5.8, 6.1]         45       0.200000
(6.1, 6.4]         30       0.033333
(6.4, 6.7]         12       0.166667
(6.7, 7.0]         10       0.200000
(7.0, 8.0]          5       0.000000
(8.0, 10.0]         2       0.500000


C:\Users\PC\AppData\Local\Temp\ipykernel_3012\996969736.py:14: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mag_grp = main_df.groupby('mag_bin')['existe_replica_fuerte'].agg(['count','mean']).rename(columns={'count':'eventos','mean':'tasa_positiva'})


,eventos,tasa_positiva
mag_bin,,
"(5.499, 5.8]",132,0.075758
"(5.8, 6.1]",45,0.200000
"(6.1, 6.4]",30,0.033333
"(6.4, 6.7]",12,0.166667
"(6.7, 7.0]",10,0.200000


In [4]:
# Etapa 2: Distribución de ventana_temporal_replica (solo casos positivos de Etapa 1)
pos_df = main_df[main_df['existe_replica_fuerte'] == 1].copy()
# Clases tempranas relevantes 1,2,3; 4 tardía; 5 inexistente (no debería estar en positivos)
stage2_counts = pos_df['ventana_temporal_replica'].value_counts().rename('count')
stage2_percent = (stage2_counts / stage2_counts.sum() * 100).round(2).rename('percent')
stage2_tbl = pd.concat([stage2_counts, stage2_percent], axis=1).sort_index()
print("Distribución Etapa 2 (solo positivos etapa 1):")
print(stage2_tbl)

# Chequear si alguna clase temprana es muy pequeña
min_temprana = stage2_counts.filter(items=[1,2,3]).min() if any(stage2_counts.index.isin([1,2,3])) else 0
print(f"\nMínimo de ejemplos en clases tempranas (1-3): {min_temprana}")
if min_temprana < 20:
    print("ADVERTENCIA: Clase con <20 ejemplos; considerar agrupar (p.ej. 2 y 3) o técnicas de balanceo")

# Distribución por zona para positivos
zona_dist = pos_df.groupby('zona_sismica')['ventana_temporal_replica'].value_counts().unstack(fill_value=0)
print("\nVentanas por zona (positivos):")
print(zona_dist)

zona_dist.head()

Distribución Etapa 2 (solo positivos etapa 1):
                          count  percent
ventana_temporal_replica                
1                            19     76.0
2                             2      8.0
3                             4     16.0

Mínimo de ejemplos en clases tempranas (1-3): 2
ADVERTENCIA: Clase con <20 ejemplos; considerar agrupar (p.ej. 2 y 3) o técnicas de balanceo

Ventanas por zona (positivos):
ventana_temporal_replica   1  2  3
zona_sismica                      
Centro                     9  2  2
Norte                     10  0  2


ventana_temporal_replica,1,2,3
zona_sismica,,,
Centro,9,2,2
Norte,10,0,2


In [5]:
# Sugerencia de técnica de balanceo según estadísticas
from collections import Counter

suggestion = []
# Etapa 1
neg = stage1_counts.get(0,0); pos = stage1_counts.get(1,0)
imbalance_ratio = (neg/pos) if pos>0 else np.inf
print(f"\nRatio desbalance etapa 1 (neg/pos): {imbalance_ratio:.2f}")
if imbalance_ratio > 3:
    suggestion.append("Etapa 1: probar class_weight y evaluar SMOTE sólo si Recall < objetivo.")
elif imbalance_ratio > 2:
    suggestion.append("Etapa 1: class_weight probablemente suficiente, ajustar umbral de probabilidad.")
else:
    suggestion.append("Etapa 1: desbalance moderado, iniciar sin técnicas de oversampling.")

# Etapa 2
if min_temprana < 20:
    suggestion.append("Etapa 2: agrupar ventanas 2 y 3 o aplicar SMOTE Borderline en la minoritaria.")
elif min_temprana < 35:
    suggestion.append("Etapa 2: considerar focal loss / class_weight para asegurar aprendizaje de clase menos frecuente.")
else:
    suggestion.append("Etapa 2: suficiente tamaño por clase, entrenar directo.")

print("\nSugerencias:")
for s in suggestion:
    print("-", s)

# Tabla resumen compacta
summary = {
    'etapa1_total_mainshocks': len(main_df),
    'etapa1_positivos': pos,
    'etapa1_negativos': neg,
    'etapa1_ratio_neg_pos': round(imbalance_ratio,2),
    'etapa2_min_clase_temprana': int(min_temprana)
}
print("\nResumen:")
print(summary)


Ratio desbalance etapa 1 (neg/pos): 8.44

Sugerencias:
- Etapa 1: probar class_weight y evaluar SMOTE sólo si Recall < objetivo.
- Etapa 2: agrupar ventanas 2 y 3 o aplicar SMOTE Borderline en la minoritaria.

Resumen:
{'etapa1_total_mainshocks': 236, 'etapa1_positivos': np.int64(25), 'etapa1_negativos': np.int64(211), 'etapa1_ratio_neg_pos': np.float64(8.44), 'etapa2_min_clase_temprana': 2}


In [6]:
# === FUSIÓN DEFINITIVA DE VENTANAS ===
# Original: 1(0-24h),2(24-48h),3(48-72h),4(>72h),5(Sin réplica)
# Nueva codificación:
# 1: 0-24h
# 2: 24-72h (fusión de originales 2 y 3)
# 3: >72h (original 4)
# 4: Sin réplica (original 5)

df_fusion = df.copy()
mapping = {
    1: 1,
    2: 2,
    3: 2,
    4: 3,
    5: 4
}
df_fusion['ventana_temporal_replica_fusion'] = df_fusion['ventana_temporal_replica'].map(mapping)

# Verificación rápida
print("Conteo original:")
print(df['ventana_temporal_replica'].value_counts().sort_index())
print("\nConteo fusionado:")
print(df_fusion['ventana_temporal_replica_fusion'].value_counts().sort_index())

# Guardar nuevo dataset
out_path = '../2da_etapa_creacion_features/seismic_features_fusion.csv'
df_fusion.to_csv(out_path, index=False)
print(f"\nArchivo guardado: {out_path}")

# Subconjunto para etapa 2 (solo clases tempranas para predicción de ventana ≤72h)
etapa2_df = df_fusion[df_fusion['ventana_temporal_replica_fusion'].isin([1,2])].copy()
print("\nDistribución etapa2 tempranas (1,2):")
print(etapa2_df['ventana_temporal_replica_fusion'].value_counts())

# Guardar subconjunto etapa 2
etapa2_df.to_csv('../2da_etapa_creacion_features/etapa2_tempranas.csv', index=False)
print("Subconjunto temprano guardado: ../2da_etapa_creacion_features/etapa2_tempranas.csv")

Conteo original:
ventana_temporal_replica
1     574
2     158
3     109
4    3023
5     151
Name: count, dtype: int64

Conteo fusionado:
ventana_temporal_replica_fusion
1     574
2     267
3    3023
4     151
Name: count, dtype: int64

Archivo guardado: ../2da_etapa_creacion_features/seismic_features_fusion.csv

Distribución etapa2 tempranas (1,2):
ventana_temporal_replica_fusion
1    574
2    267
Name: count, dtype: int64
Subconjunto temprano guardado: ../2da_etapa_creacion_features/etapa2_tempranas.csv


In [7]:
# === REEMPLAZO DEFINITIVO DE LA COLUMNA DE VENTANAS ===
# Partimos de df_fusion ya creado con ventana_temporal_replica_fusion (1–4)

# Validar que la fusionada solo contiene 1–4
vals_fusion = sorted(df_fusion['ventana_temporal_replica_fusion'].dropna().unique().tolist())
print("Valores únicos fusionados:", vals_fusion)
assert set(vals_fusion).issubset({1,2,3,4}), "La columna fusionada tiene valores fuera de 1–4."

# Eliminar la columna antigua
if 'ventana_temporal_replica' in df_fusion.columns:
    df_fusion = df_fusion.drop(columns=['ventana_temporal_replica'])
    print("Columna original eliminada.")

# Renombrar la fusionada a nombre estándar
df_fusion = df_fusion.rename(columns={'ventana_temporal_replica_fusion': 'ventana_temporal_replica'})

# Guardar dataset definitivo
out_final = '../2da_etapa_creacion_features/seismic_features_fusion_final.csv'
df_fusion.to_csv(out_final, index=False)
print(f"Archivo final guardado: {out_final}")

# Mostrar verificación final
print("Valores finales de ventana_temporal_replica:", sorted(df_fusion['ventana_temporal_replica'].unique()))
print("Shape final:", df_fusion.shape)

Valores únicos fusionados: [1, 2, 3, 4]
Columna original eliminada.
Archivo final guardado: ../2da_etapa_creacion_features/seismic_features_fusion_final.csv
Valores finales de ventana_temporal_replica: [np.int64(1), np.int64(2), np.int64(3), np.int64(4)]
Shape final: (4015, 29)


In [8]:
# === VERIFICAR RATIO ETAPA 2 (FUSIÓN 1 vs 2)  ===
etapa2_train_df = df_fusion[
    (df_fusion['es_mainshock'] == 1) &
    (df_fusion['existe_replica_fuerte'] == 1) &
    (df_fusion['ventana_temporal_replica'].isin([1,2]))
].copy()

dist = etapa2_train_df['ventana_temporal_replica'].value_counts()
print("Distribución filtrada etapa 2 (1=0-24h, 2=24-72h):")
print(dist)

major = dist.idxmax()
minor = dist.idxmin()
ratio = dist[major] / dist[minor]
print(f"Ratio: {ratio:.2f}")

if (ratio > 2.5) or (dist[minor] < 30):
    print("Sugerencia: aplicar oversampling simple (réplica) antes de entrenar.")
    target_min = int(dist[major] / 2)  # objetivo aproximado ratio ~2:1
    need = max(0, target_min - dist[minor])
    if need > 0:
        extra = etapa2_train_df[etapa2_train_df['ventana_temporal_replica'] == minor].sample(
            n=need, replace=True, random_state=42
        )
        etapa2_balanced = pd.concat([etapa2_train_df, extra], ignore_index=True)
    else:
        etapa2_balanced = etapa2_train_df.copy()
    dist_bal = etapa2_balanced['ventana_temporal_replica'].value_counts()
    new_ratio = dist_bal[major] / dist_bal[minor]
    print("Distribución tras oversampling simple:")
    print(dist_bal)
    print(f"Nuevo ratio: {new_ratio:.2f}")
    etapa2_balanced.to_csv('../2da_etapa_creacion_features/etapa2_train_balanced.csv', index=False)
    print("Guardado: ../2da_etapa_creacion_features/etapa2_train_balanced.csv")
else:
    print("No oversampling necesario ahora; usar class_weight en el modelo.")
    etapa2_train_df.to_csv('../2da_etapa_creacion_features/etapa2_train_filtrado.csv', index=False)
    print("Guardado: ../2da_etapa_creacion_features/etapa2_train_filtrado.csv")

Distribución filtrada etapa 2 (1=0-24h, 2=24-72h):
ventana_temporal_replica
1    19
2     6
Name: count, dtype: int64
Ratio: 3.17
Sugerencia: aplicar oversampling simple (réplica) antes de entrenar.
Distribución tras oversampling simple:
ventana_temporal_replica
1    19
2     9
Name: count, dtype: int64
Nuevo ratio: 2.11
Guardado: ../2da_etapa_creacion_features/etapa2_train_balanced.csv


In [9]:
path_filtrado = 'etapa2_train_filtrado.csv'
etapa2_train_df.to_csv(path_filtrado, index=False)
print(f"Guardado filtrado: {path_filtrado}")

Guardado filtrado: etapa2_train_filtrado.csv


(25, 29)